In [70]:
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display

# Function to calculate Awesome Oscillator
def calculate_ao(data, short_window=3, long_window=15, price_col='Close'):
    if price_col not in data.columns:
        raise KeyError(f"Column '{price_col}' not found in DataFrame")
    data['SMA_short'] = data[price_col].rolling(window=short_window).mean()
    data['SMA_long'] = data[price_col].rolling(window=long_window).mean()
    data['AO'] = data['SMA_short'] - data['SMA_long']
    return data

# Function to calculate RSI
def calculate_rsi(data, periods=14, price_col='Close'):
    if price_col not in data.columns:
        raise KeyError(f"Column '{price_col}' not found in DataFrame")
    delta = data[price_col].diff()
    up = delta.clip(lower=0)
    down = (-delta).clip(lower=0)
    rs = up.ewm(com=periods-1, adjust=False).mean() / down.ewm(com=periods-1, adjust=False).mean()
    data['RSI'] = 100 - 100 / (1 + rs)
    return data

# Define the tickers for Bitcoin and Ethereum
tickers = ['BTC-USD', 'ETH-USD']

# Define the date range
start_date = '2023-04-01'
end_date = '2025-04-01'

# Download daily data and calculate indicators
daily_data = {}
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
for ticker in tickers:
    try:
        df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
        if df.empty:
            raise ValueError(f"No data returned for {ticker}")

        # Handle multi-index columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

        # Standardize column names
        column_map = {col.lower(): col.capitalize() for col in df.columns}
        df = df.rename(columns=column_map)

        # Verify required columns
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise KeyError(f"Missing columns for {ticker}: {missing_cols}")

        daily_data[ticker] = df

        # Calculate AO and RSI, then drop NaN values
        daily_data[ticker] = calculate_ao(daily_data[ticker])
        daily_data[ticker] = calculate_rsi(daily_data[ticker])
        daily_data[ticker] = daily_data[ticker].dropna()
    except Exception as e:
        print(f"Error fetching daily data for {ticker}: {e}")
        continue

# Aggregate daily data into 10-day periods
ten_day_data = {}
for ticker in daily_data:
    try:
        df = daily_data[ticker].copy()

        # Create a period index for 10-day intervals
        df['Period'] = (df.index - pd.Timestamp(start_date)).days // 10

        # Group by period and aggregate
        aggregated = df.groupby('Period').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        })

        # Convert period back to dates (last day of each 10-day period)
        aggregated.index = pd.Timestamp(start_date) + pd.to_timedelta(aggregated.index * 10 + 9, unit='D')
        ten_day_data[ticker] = aggregated

        # Calculate AO and RSI, then drop NaN values
        ten_day_data[ticker] = calculate_ao(ten_day_data[ticker])
        ten_day_data[ticker] = calculate_rsi(ten_day_data[ticker])
        ten_day_data[ticker] = ten_day_data[ticker].dropna()
    except Exception as e:
        print(f"Error aggregating 10-day data for {ticker}: {e}")
        continue



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [75]:
# Function to simulate the improved trading strategy
def simulate_strategy(data, timeframe_name):
    data = data.copy()
    position = None
    trades = []
    stop_loss = 0.05  # 5% stop-loss

    for i in range(1, len(data)):
        current_price = data['Close'].iloc[i]
        if position is None:
            # Long Entry: AO > 0 and RSI < 50
            if data['AO'].iloc[i] > 0 and data['RSI'].iloc[i] < 70:
                position = 'long'
                entry_price = current_price
                entry_date = data.index[i]
            # Short Entry: AO < 0 and RSI > 50
            elif data['AO'].iloc[i] < 0 and data['RSI'].iloc[i] > 30:
                position = 'short'
                entry_price = current_price
                entry_date = data.index[i]
        elif position == 'long':
            # Long Exit: RSI > 65 or AO[i] < AO[i-1] or stop-loss
            if (data['RSI'].iloc[i] > 65 or
                data['AO'].iloc[i] < data['AO'].iloc[i-1] or
                current_price < entry_price * (1 - stop_loss)):
                exit_price = current_price
                exit_date = data.index[i]
                profit = exit_price - entry_price
                trades.append({
                    'entry_date': entry_date,
                    'exit_date': exit_date,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit': profit,
                    'type': 'long'
                })
                position = None
        elif position == 'short':
            # Short Exit: RSI < 35 or AO[i] > AO[i-1] or stop-loss
            if (data['RSI'].iloc[i] < 35 or
                data['AO'].iloc[i] > data['AO'].iloc[i-1] or
                current_price > entry_price * (1 + stop_loss)):
                exit_price = current_price
                exit_date = data.index[i]
                profit = entry_price - exit_price
                trades.append({
                    'entry_date': entry_date,
                    'exit_date': exit_date,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit': profit,
                    'type': 'short'
                })
                position = None

    return pd.DataFrame(trades)


In [76]:
# Function to perform backtest
def backtest_strategy(trades, data, initial_capital=1000, R=2, S=0.05):
    if trades.empty:
        return {
            'net_profit': 0,
            'max_drawdown': 0,
            'sharpe_ratio': 0,
            'sortino_ratio': 0,
            'portfolio_values': pd.Series()
        }

    cash = initial_capital
    shares = 0
    portfolio_values = []
    position = None
    entry_price = None

    for date in data.index:
        current_price = data.loc[date, 'Close']

        # Calculate current portfolio value
        V = cash + shares * current_price

        # Check for entry
        if position is None:
            entry_trades = trades[trades['entry_date'] == date]
            if not entry_trades.empty:
                trade = entry_trades.iloc[0]
                position = trade['type']
                entry_price = trade['entry_price']

                # Calculate position size P based on risk
                P = (V * R / 100) / S

                if position == 'long':
                    if P > cash:
                        P = cash
                    shares = P / current_price
                    cash -= P
                elif position == 'short':
                    shares = - (P / current_price)
                    cash += P

        # Check for exit
        elif position is not None:
            exit_trades = trades[trades['exit_date'] == date]
            if not exit_trades.empty:
                # Exit the position
                cash += shares * current_price
                shares = 0
                position = None
                entry_price = None

        # Calculate current portfolio value
        current_value = cash + shares * current_price
        portfolio_values.append(current_value)

    portfolio_values = pd.Series(portfolio_values, index=data.index)

    # Calculate returns
    returns = portfolio_values.pct_change().dropna()

    # Net profit
    net_profit = portfolio_values.iloc[-1] - initial_capital

    # Max drawdown
    rolling_max = portfolio_values.cummax()
    drawdowns = (portfolio_values - rolling_max) / rolling_max
    max_drawdown = drawdowns.min()

    # Sharpe ratio (assuming risk-free rate = 0)
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252) if returns.std() != 0 else 0

    # Sortino ratio (assuming risk-free rate = 0)
    downside_returns = returns[returns < 0]
    sortino_ratio = returns.mean() / downside_returns.std() * np.sqrt(252) if downside_returns.std() != 0 else 0

    return {
        'net_profit': net_profit,
        'max_drawdown': max_drawdown,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'portfolio_values': portfolio_values
    }

In [77]:
# Apply strategy and backtest to both timeframes for each ticker
for ticker in tickers:
    if ticker in daily_data and not daily_data[ticker].empty:
        trades_daily = simulate_strategy(daily_data[ticker], 'daily')
        backtest_daily = backtest_strategy(trades_daily, daily_data[ticker])
        trade_count_daily = len(trades_daily)
        print(f"\nBacktest Results for {ticker}_daily (Trades: {trade_count_daily}):")
        if trade_count_daily > 0:
            print(f"Net Profit: ${backtest_daily['net_profit']:.2f}")
            print(f"Max Drawdown: {backtest_daily['max_drawdown']:.4f}")
            print(f"Sharpe Ratio: {backtest_daily['sharpe_ratio']:.4f}")
            print(f"Sortino Ratio: {backtest_daily['sortino_ratio']:.4f}")
            display(trades_daily)
        else:
            print("No trades executed")
    else:
        print(f"No valid daily data for {ticker}")

    if ticker in ten_day_data and not ten_day_data[ticker].empty:
        trades_10day = simulate_strategy(ten_day_data[ticker], '10day')
        backtest_10day = backtest_strategy(trades_10day, ten_day_data[ticker])
        trade_count_10day = len(trades_10day)
        print(f"\nBacktest Results for {ticker}_10day (Trades: {trade_count_10day}):")
        if trade_count_10day > 0:
            print(f"Net Profit: ${backtest_10day['net_profit']:.2f}")
            print(f"Max Drawdown: {backtest_10day['max_drawdown']:.4f}")
            print(f"Sharpe Ratio: {backtest_10day['sharpe_ratio']:.4f}")
            print(f"Sortino Ratio: {backtest_10day['sortino_ratio']:.4f}")
            display(trades_10day)
        else:
            print("No trades executed")
    else:
        print(f"No valid 10-day data for {ticker}")


Backtest Results for BTC-USD_daily (Trades: 235):
Net Profit: $-71.73
Max Drawdown: -0.1967
Sharpe Ratio: -0.1774
Sortino Ratio: -0.1862


,entry_date,exit_date,entry_price,exit_price,profit,type
0,2023-04-16,2023-04-17,30315.355469,29445.044922,-870.310547,long
1,2023-04-18,2023-04-19,30397.552734,28822.679688,-1574.873047,long
2,2023-04-20,2023-04-21,28245.988281,27276.910156,969.078125,short
3,2023-04-22,2023-04-24,27817.500000,27525.339844,292.160156,short
4,2023-04-25,2023-04-26,28307.597656,28422.701172,-115.103516,short
...,...,...,...,...,...,...
230,2025-03-16,2025-03-17,82579.687500,84075.687500,-1496.000000,short
231,2025-03-18,2025-03-19,82718.500000,86854.226562,-4135.726562,short
232,2025-03-20,2025-03-22,84167.195312,83832.484375,-334.710938,long
233,2025-03-23,2025-03-27,86054.375000,87177.101562,1122.726562,long



Backtest Results for BTC-USD_10day (Trades: 13):
Net Profit: $35.09
Max Drawdown: -0.0796
Sharpe Ratio: 0.7595
Sortino Ratio: 0.6606


,entry_date,exit_date,entry_price,exit_price,profit,type
0,2023-09-17,2023-09-27,26534.187500,26352.716797,181.470703,short
1,2023-10-07,2023-10-17,27968.839844,28415.748047,-446.908203,short
2,2024-01-25,2024-02-04,39933.808594,42583.582031,2649.773438,long
3,2024-05-14,2024-05-24,61552.789062,68526.101562,6973.312500,long
4,2024-06-23,2024-07-03,63180.796875,60173.921875,-3006.875000,long
5,2024-07-13,2024-07-23,59231.953125,65927.671875,-6695.718750,short
6,2024-08-02,2024-08-12,61415.066406,59354.515625,2060.550781,short
7,2024-08-22,2024-09-21,60381.914062,63394.839844,-3012.925781,short
8,2024-10-01,2024-10-11,60837.007812,62445.089844,-1608.082031,short
9,2024-10-21,2024-11-10,67367.851562,80474.187500,13106.335938,long



Backtest Results for ETH-USD_daily (Trades: 244):
Net Profit: $23.23
Max Drawdown: -0.1850
Sharpe Ratio: 0.1284
Sortino Ratio: 0.1518


,entry_date,exit_date,entry_price,exit_price,profit,type
0,2023-04-16,2023-04-17,2120.005859,2076.242920,-43.762939,long
1,2023-04-18,2023-04-19,2104.537354,1936.403442,-168.133911,long
2,2023-04-20,2023-04-21,1943.097656,1849.999878,-93.097778,long
3,2023-04-22,2023-04-25,1874.228638,1866.753662,7.474976,short
4,2023-04-26,2023-04-27,1866.564209,1908.786377,-42.222168,short
...,...,...,...,...,...,...
239,2025-03-19,2025-03-20,2057.749023,1982.099854,75.649170,short
240,2025-03-21,2025-03-22,1964.847534,1980.037842,15.190308,long
241,2025-03-23,2025-03-26,2005.299194,2009.189331,3.890137,long
242,2025-03-27,2025-03-28,2002.357422,1895.502930,-106.854492,long



Backtest Results for ETH-USD_10day (Trades: 19):
Net Profit: $293.38
Max Drawdown: -0.0824
Sharpe Ratio: 3.1370
Sortino Ratio: 5.9513


,entry_date,exit_date,entry_price,exit_price,profit,type
0,2023-10-27,2023-11-06,1780.045288,1899.837402,-119.792114,short
1,2023-11-16,2024-01-05,1960.881592,2268.647217,307.765625,long
2,2024-01-25,2024-02-04,2217.710205,2289.546143,71.835938,long
3,2024-02-14,2024-02-24,2777.902344,2992.385986,214.483643,long
4,2024-04-04,2024-04-14,3330.040527,3156.941895,-173.098633,long
5,2024-04-24,2024-05-04,3139.805176,3117.576416,-22.228760,long
6,2024-05-14,2024-05-24,2881.157959,3726.934570,845.776611,long
7,2024-06-03,2024-06-13,3766.389160,3469.281250,-297.107910,long
8,2024-06-23,2024-07-03,3418.611816,3292.916748,-125.695068,long
9,2024-07-13,2024-07-23,3177.198730,3481.995850,-304.797119,short
